In [2]:
# import py7zr
# from pathlib import Path

# # extract compressed data file
# data_path = Path('data')
# with py7zr.SevenZipFile(data_path.joinpath('data_dirty.7z'), mode='r') as z:
#     z.extractall(data_path)

In [1]:
import io
import re
import string
import tqdm

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

In [2]:
%load_ext tensorboard

In [3]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

In [4]:
sentence = "The wide road shimmered in the hot sun"
tokens = list(sentence.lower().split())
print(len(tokens))

8
